In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
import pandas as pd
import math
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import json

In [8]:
data = pd.read_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5.csv')
data
# 카페 데이터만
data = data.loc[data['상권업종중분류명'] == '커피점/카페']  
data

,Unnamed: 0,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,...,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도,역명,거리,카카오url
7,60,28490990,406번째스토브,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,158050,7999.0,NaN,NaN,NaN,126.874168,37.525177,5호선 오목교(목동운동장앞),42.783155,NaN
8,81,28491095,우성커피숍,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,158090,8029.0,NaN,1,NaN,126.842935,37.522055,5호선 까치산,794.156052,NaN
10,96,23260840,버블베어,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,157788,7509.0,NaN,1,NaN,126.813358,37.580941,5호선 방화,261.092137,NaN
55,388,25337952,리버스로스터,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,158090,7909.0,NaN,1,NaN,126.823946,37.535437,5호선 마곡,1658.491978,NaN
61,406,27535314,필링,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,110320,3139.0,NaN,1,하,126.989443,37.570993,5호선 종로3가(탑골공원),125.480924,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20025,135714,17207878,세라젬웰카페,영등포점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,150030,7301.0,NaN,1,NaN,126.904491,37.519552,5호선 영등포시장,215.460909,NaN
20026,135716,17207899,미타임,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,133783,4806.0,NaN,1,NaN,127.057222,37.562579,5호선 답십리,587.455177,NaN
20027,135717,17207763,휴식상점,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,130843,2634.0,NaN,1,3,127.062036,37.563239,5호선 장한평,290.438891,NaN
20043,135797,17211625,단백한하우스,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,138812,5782.0,NaN,1,101,127.134560,37.493511,5호선 개롱,357.124389,NaN


In [10]:
data = data[["상호명","상권업종중분류명","상권업종소분류명","도로명주소","역명","거리","카카오url"]]
data["역"] = data["역명"].apply(lambda x : x.split(" ")[1])
data["역"] = data["역"].apply(lambda x : x.split("(")[0])
data["검색어"] = data.apply(lambda x : x["역"] + "역 " + str(x["상호명"]),axis=1)
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
7,406번째스토브,커피점/카페,커피전문점/카페/다방,서울특별시 양천구 오목로 329-6,5호선 오목교(목동운동장앞),42.783155,NaN,오목교,오목교역 406번째스토브
8,우성커피숍,커피점/카페,커피전문점/카페/다방,서울특별시 양천구 오목로 48,5호선 까치산,794.156052,NaN,까치산,까치산역 우성커피숍
10,버블베어,커피점/카페,커피전문점/카페/다방,서울특별시 강서구 금낭화로 167,5호선 방화,261.092137,NaN,방화,방화역 버블베어
55,리버스로스터,커피점/카페,커피전문점/카페/다방,서울특별시 양천구 가로공원로 71,5호선 마곡,1658.491978,NaN,마곡,마곡역 리버스로스터
61,필링,커피점/카페,커피전문점/카페/다방,서울특별시 종로구 수표로 110,5호선 종로3가(탑골공원),125.480924,NaN,종로3가,종로3가역 필링
...,...,...,...,...,...,...,...,...,...
20025,세라젬웰카페,커피점/카페,커피전문점/카페/다방,서울특별시 영등포구 영등포로 210,5호선 영등포시장,215.460909,NaN,영등포시장,영등포시장역 세라젬웰카페
20026,미타임,커피점/카페,커피전문점/카페/다방,서울특별시 성동구 천호대로 346,5호선 답십리,587.455177,NaN,답십리,답십리역 미타임
20027,휴식상점,커피점/카페,커피전문점/카페/다방,서울특별시 동대문구 한천로6길 47-2,5호선 장한평,290.438891,NaN,장한평,장한평역 휴식상점
20043,단백한하우스,커피점/카페,커피전문점/카페/다방,서울특별시 송파구 문정로11길 12,5호선 개롱,357.124389,NaN,개롱,개롱역 단백한하우스


In [11]:
items = data["검색어"][2518:]
items

16913        을지로4가역 이디야커피
16917           군자역 이디야커피
16926    강동역 Coffee3Times
16936       광화문역 달콤커피광화문점
16949          까치산역 어울림카페
               ...       
20025       영등포시장역 세라젬웰카페
20026            답십리역 미타임
20027           장한평역 휴식상점
20043          개롱역 단백한하우스
20044            오금역 만랩커피
Name: 검색어, Length: 603, dtype: object

In [12]:
rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])
img_elem = pd.DataFrame(columns = ["ItemID","img_url"])
review_elem = pd.DataFrame(columns = ["ItemID", "review"])
user_review_id = {}
review_json = {}
image_json = {}

In [13]:
# 드라이버 구동
driver = webdriver.Chrome(executable_path='C:/Users/정예림/Desktop/추천시스템 프로젝트/chromedriver.exe')
# 창 크기 최대화
driver.maximize_window() 



count = 0
current = 0
indexing = 0
goal = len(items)
for item in items :
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    # 리뷰가 없을 때의 코드
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    time.sleep(2)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    if len(driver.find_elements_by_xpath("//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href") 
        #상세보기 URL수집
        url = driver.find_element_by_xpath('//*[@id="info.search.place.list"]/li[1]/div[5]/div[4]/a[1]').get_attribute('href')
        data.iloc[indexing,-3] = url
        print(url)
        tabs = driver.window_handles
        driver.switch_to_window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(3)
        html = driver.page_source
        time.sleep(20)
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        print(len(review_lists))
        if len(review_lists) != 0 :
            for i, review in enumerate(review_lists) :
                user_review = review.select('.txt_comment > span') # 리뷰
                rating = review.select('.grade_star > em') # 별점
                img_url = review.select('.link_photo > img') #이미지url
                user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                timestamp = review.select(' div > span.time_write') #시간정보
                try:
                    row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_ur}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)

                    time.sleep(1)
                except:
                    row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_url}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    time.sleep(1)

                
                    
        else :
            print("리뷰가 없습니다")
            
        try:
            for i in range(2,500):
                time.sleep(3)
                another_review = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                another_review.click()
                time.sleep(3)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                review_lists = soup.select('.list_evaluation > li')
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.txt_comment > span') # 리뷰
                        rating = review.select('.grade_star > em') # 별점
                        img_url = review.select('.link_photo > img') #이미지url
                        user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                        timestamp = review.select(' div > span.time_write') #시간정보
                        try:
                            row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                            
                        except:
                            row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                        
                        
                            
        except:
            print("더 이상 리뷰 존재 X")
            driver.close()
        driver.switch_to_window(tabs[0])
        print("기본 페이지로 돌아가자")
            
    else:
        data.iloc[indexing,-3] = None
        print("식당 존재 x")
    
    indexing += 1


진행상황 : 1/603
식당 존재
https://place.map.kakao.com/146241342
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2/603
식당 존재
https://place.map.kakao.com/15625476
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 3/603
식당 존재
https://place.map.kakao.com/1080522567
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 4/603
식당 존재 x
진행상황 : 5/603
식당 존재 x
진행상황 : 6/603
식당 존재
https://place.map.kakao.com/10021760
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 7/603
식당 존재 x
진행상황 : 8/603
식당 존재
https://place.map.kakao.com/20556173
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 9/603
식당 존재
https://place.map.kakao.com/9631217
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 10/603
식당 존재 x
진행상황 : 11/603
식당 존재
https://place.map.kakao.com/679173861
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 12/603
식당 존재
https://place.map.kakao.com/441732027
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 13/603
식당 존재 x
진행상황 : 14/603
식당 존재 x
진행상황 : 15/603
식당 존재
https://place.map.kakao.com/20046342
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 16/603
식당 존재
https://place.map.kakao.com/1662602781
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상

식당 존재
https://place.map.kakao.com/15864607
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 128/603
식당 존재
https://place.map.kakao.com/18320406
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 129/603
식당 존재
https://place.map.kakao.com/1794056893
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 130/603
식당 존재
https://place.map.kakao.com/7968246
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 131/603
식당 존재
https://place.map.kakao.com/21528728
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 132/603
식당 존재
https://place.map.kakao.com/8047272
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 133/603
식당 존재
https://place.map.kakao.com/210128075
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 134/603
식당 존재
https://place.map.kakao.com/568354371
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 135/603
식당 존재 x
진행상황 : 136/603
식당 존재
https://place.map.kakao.com/2016685130
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 137/603
식당 존재 x
진행상황 : 138/603
식당 존재
https://place.map.kakao.com/19293558
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 139/603
식당 존재
https://place.map.kakao.com/78911659
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 140

식당 존재
https://place.map.kakao.com/635670307
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 243/603
식당 존재
https://place.map.kakao.com/1107140729
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 244/603
식당 존재
https://place.map.kakao.com/1694586322
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 245/603
식당 존재 x
진행상황 : 246/603
식당 존재 x
진행상황 : 247/603
식당 존재
https://place.map.kakao.com/14518298
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 248/603
식당 존재
https://place.map.kakao.com/498069706
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 249/603
식당 존재
https://place.map.kakao.com/15625476
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 250/603
식당 존재 x
진행상황 : 251/603
식당 존재
https://place.map.kakao.com/25412196
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 252/603
식당 존재 x
진행상황 : 253/603
식당 존재 x
진행상황 : 254/603
식당 존재 x
진행상황 : 255/603
식당 존재
https://place.map.kakao.com/16909845
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 256/603
식당 존재
https://place.map.kakao.com/19034301
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 257/603
식당 존재 x
진행상황 : 258/603
식당 존재
https://

식당 존재
https://place.map.kakao.com/1728663678
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 352/603
식당 존재 x
진행상황 : 353/603
식당 존재
https://place.map.kakao.com/1524882023
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 354/603
식당 존재
https://place.map.kakao.com/602718974
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 355/603
식당 존재
https://place.map.kakao.com/1164293189
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 356/603
식당 존재
https://place.map.kakao.com/932797688
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 357/603
식당 존재
https://place.map.kakao.com/524830559
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 358/603
식당 존재
https://place.map.kakao.com/1699763784
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 359/603
식당 존재 x
진행상황 : 360/603
식당 존재 x
진행상황 : 361/603
식당 존재 x
진행상황 : 362/603
식당 존재
https://place.map.kakao.com/316085205
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 363/603
식당 존재
https://place.map.kakao.com/1162455471
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 364/603
식당 존재 x
진행상황 : 365/603
식당 존재 x
진행상황 : 366/603
식당 존재
https://place.map.kakao.com/669270382
5
더

5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 467/603
식당 존재
https://place.map.kakao.com/32483834
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 468/603
식당 존재 x
진행상황 : 469/603
식당 존재
https://place.map.kakao.com/1527861014
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 470/603
식당 존재 x
진행상황 : 471/603
식당 존재 x
진행상황 : 472/603
식당 존재 x
진행상황 : 473/603
식당 존재
https://place.map.kakao.com/687704338
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 474/603
식당 존재
https://place.map.kakao.com/1661945453
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 475/603
식당 존재
https://place.map.kakao.com/1727798216
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 476/603
식당 존재 x
진행상황 : 477/603
식당 존재
https://place.map.kakao.com/576197946
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 478/603
식당 존재 x
진행상황 : 479/603
식당 존재
https://place.map.kakao.com/1969326334
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 480/603
식당 존재
https://place.map.kakao.com/1292595476
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 481/603
식당 존재
https://place.map.kakao.com/220650222
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행

식당 존재
https://place.map.kakao.com/308778812
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 583/603
식당 존재
https://place.map.kakao.com/9796782
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 584/603
식당 존재
https://place.map.kakao.com/524830559
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 585/603
식당 존재
https://place.map.kakao.com/1973568744
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 586/603
식당 존재
https://place.map.kakao.com/1942818618
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 587/603
식당 존재
https://place.map.kakao.com/247663431
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 588/603
식당 존재 x
진행상황 : 589/603
식당 존재
https://place.map.kakao.com/206172834
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 590/603
식당 존재
https://place.map.kakao.com/215379134
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 591/603
식당 존재
https://place.map.kakao.com/977211586
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 592/603
식당 존재
https://place.map.kakao.com/27344415
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 593/603
식당 존재
https://place.map.kakao.com/812616608
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 59

In [14]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,qkg4u8,을지로4가역 이디야커피,1점,2020.10.22.
1,None,을지로4가역 이디야커피,None,2018.07.19.
2,None,을지로4가역 이디야커피,None,2018.04.17.
3,None,을지로4가역 이디야커피,None,2018.01.18.
4,1dgkle7,군자역 이디야커피,5점,2020.11.10.
...,...,...,...,...
2701,1041b28,명일역 투썸플레이스,5점,2021.01.22.
2702,umsto2,명일역 투썸플레이스,5점,2020.12.28.
2703,9m27hn,명일역 투썸플레이스,5점,2020.12.25.
2704,tg75pd,영등포시장역 세라젬웰카페,5점,2021.03.14.


In [15]:
img_elem

,ItemID,img_url
0,을지로4가역 이디야커피,None
1,을지로4가역 이디야커피,None
2,을지로4가역 이디야커피,None
3,을지로4가역 이디야커피,None
4,군자역 이디야커피,None
...,...,...
2701,명일역 투썸플레이스,None
2702,명일역 투썸플레이스,None
2703,명일역 투썸플레이스,None
2704,영등포시장역 세라젬웰카페,None


In [16]:
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
7,406번째스토브,커피점/카페,커피전문점/카페/다방,서울특별시 양천구 오목로 329-6,5호선 오목교(목동운동장앞),42.783155,https://place.map.kakao.com/146241342,오목교,오목교역 406번째스토브
8,우성커피숍,커피점/카페,커피전문점/카페/다방,서울특별시 양천구 오목로 48,5호선 까치산,794.156052,https://place.map.kakao.com/15625476,까치산,까치산역 우성커피숍
10,버블베어,커피점/카페,커피전문점/카페/다방,서울특별시 강서구 금낭화로 167,5호선 방화,261.092137,https://place.map.kakao.com/1080522567,방화,방화역 버블베어
55,리버스로스터,커피점/카페,커피전문점/카페/다방,서울특별시 양천구 가로공원로 71,5호선 마곡,1658.491978,None,마곡,마곡역 리버스로스터
61,필링,커피점/카페,커피전문점/카페/다방,서울특별시 종로구 수표로 110,5호선 종로3가(탑골공원),125.480924,None,종로3가,종로3가역 필링
...,...,...,...,...,...,...,...,...,...
20025,세라젬웰카페,커피점/카페,커피전문점/카페/다방,서울특별시 영등포구 영등포로 210,5호선 영등포시장,215.460909,NaN,영등포시장,영등포시장역 세라젬웰카페
20026,미타임,커피점/카페,커피전문점/카페/다방,서울특별시 성동구 천호대로 346,5호선 답십리,587.455177,NaN,답십리,답십리역 미타임
20027,휴식상점,커피점/카페,커피전문점/카페/다방,서울특별시 동대문구 한천로6길 47-2,5호선 장한평,290.438891,NaN,장한평,장한평역 휴식상점
20043,단백한하우스,커피점/카페,커피전문점/카페/다방,서울특별시 송파구 문정로11길 12,5호선 개롱,357.124389,NaN,개롱,개롱역 단백한하우스


In [17]:
img_elem

,ItemID,img_url
0,을지로4가역 이디야커피,None
1,을지로4가역 이디야커피,None
2,을지로4가역 이디야커피,None
3,을지로4가역 이디야커피,None
4,군자역 이디야커피,None
...,...,...
2701,명일역 투썸플레이스,None
2702,명일역 투썸플레이스,None
2703,명일역 투썸플레이스,None
2704,영등포시장역 세라젬웰카페,None


In [18]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,qkg4u8,을지로4가역 이디야커피,1점,2020.10.22.
1,None,을지로4가역 이디야커피,None,2018.07.19.
2,None,을지로4가역 이디야커피,None,2018.04.17.
3,None,을지로4가역 이디야커피,None,2018.01.18.
4,1dgkle7,군자역 이디야커피,5점,2020.11.10.
...,...,...,...,...
2701,1041b28,명일역 투썸플레이스,5점,2021.01.22.
2702,umsto2,명일역 투썸플레이스,5점,2020.12.28.
2703,9m27hn,명일역 투썸플레이스,5점,2020.12.25.
2704,tg75pd,영등포시장역 세라젬웰카페,5점,2021.03.14.


In [19]:
review_elem

,ItemID,review
0,을지로4가역 이디야커피,바빠서그런건지 그냥 짜증이 나신건지 불친절 인근에 갈만한 커피숍이 여기뿐
1,을지로4가역 이디야커피,냉면먹고 갈곳이 인근.시원한.커피숍이 여기밖에 ㅋ 자리.꽉참
2,을지로4가역 이디야커피,냉면먹고 간단히 차한잔하기 좋은곳
3,을지로4가역 이디야커피,커피 맛있고 친절합니다
4,군자역 이디야커피,아침마다 요즘 도장 찍는곳 맛있고 친절한 매장!
...,...,...
2701,명일역 투썸플레이스,직원 분들도 친절하고 케이크 이뻐요 맛있고 굿굿
2702,명일역 투썸플레이스,원두 종류도 고를수있고 케익도 너무 맛있어요
2703,명일역 투썸플레이스,최고예요
2704,영등포시장역 세라젬웰카페,직원분이 친절하게 설명도 잘해주시고 음료도 맛있게 먹었어요~


In [22]:
img_elem.head()

,ItemID,img_url
0,을지로4가역 이디야커피,None
1,을지로4가역 이디야커피,None
2,을지로4가역 이디야커피,None
3,을지로4가역 이디야커피,None
4,군자역 이디야커피,None


In [26]:
ItemID_elem = rating_df['ItemID'].unique()

for k in range(len(ItemID_elem)) :
    User_ID_elem = rating_df.loc[rating_df['ItemID'] == ItemID_elem[k]]
    Uder_ID_elem = User_ID_elem[['UserID','ItemID']]
print(User_ID_elem)





ItemID_elem = img_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_img_elem = img_elem.loc[img_elem['ItemID'] == ItemID_elem[j]]
print(item_img_elem)



ItemID_elem = review_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_review_elem = review_elem.loc[review_elem['ItemID'] == ItemID_elem[j]]
print(item_review_elem)




       UserID    ItemID Rating    Timestamp
2705  1fa28s5  오금역 만랩커피     5점  2021.04.11.
        ItemID img_url
2705  오금역 만랩커피    None
        ItemID review
2705  오금역 만랩커피    좋아유


In [27]:
rating_df.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_cafe(final).csv')

In [28]:
User_ID_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_cafe_User(final).csv')
item_img_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_cafe_img(final).csv')
item_review_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/lline5_cafe_review(final).csv')

In [29]:
ItemID_elem = rating_df['ItemID'].unique()
len(ItemID_elem)


267

In [31]:
data.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_(final)url.csv')